<a href="https://colab.research.google.com/github/acedesci/scanalytics/blob/master/EN/S06_Time-Series_Analytics/Demo_Time_Series_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo: Time-series demand forecasting pipeline

## Introduction

Time series demand forecasting is crucial for effective supply chain management. With accurate demand forecasts, businesses can optimize inventory, improve demand planning, enhance sales forecasting, and mitigate supply chain risks.

With recent development in Python and opensources, there are many simple-to-use packages such as [statsmodels](https://www.statsmodels.org/stable/index.html) (mainly for statistical techniques), [Prophet (by Facebook)](https://facebook.github.io/prophet/), [GluonTS (by Amazon)] (https://ts.gluon.ai/stable/), and many other libraries. Most libraries require a specific (but similar) data input format and processes. Thus, the most important step to use such packages is to prepare the data in the right format.

There are also opensource libaries that are built upon many time-series forecasting packages and provide interfaces to many time-series algorithms such as [sktime](https://www.sktime.net/en/latest/index.html) and [darts](https://unit8co.github.io/darts/). These time-series forecasting interfaces greatly simplify forecasting with its intuitive API and diverse range of models, including classical statistical methods and modern machine learning approaches including deep-learning-based models. An illustration of a time-series pipeline based on the case [1] is depicted below. Note that, in the case, both time-series-based (which requires time-series data format) and machine-learning-based (which requires tabular data format) models are used simultaneously whereas this notebook focuses on the use of time-series models using time-series data format.

<figure>
<center>
<img src='https://github.com/acedesci/scanalytics/blob/master/EN/S06_Time-Series_Analytics/BBD_forecasts.png?raw=true'/>
<figcaption>Forecasting pipeline as depicted in [1]</figcaption></center>
</figure>

This notebook provide a simple walkthrough for an implementation of time-series forecasting pipeline using `darts`. At the end of the notebook, it also demonstrates time-phased dynamic inventory policy that can be calculated from the forecasts from various forecasting models. The quality of this inventory policy can then be measured through a simulation which has been covered in an earlier session on `DataFrame`.

[1] Dodin, P., Xiao, J., Adulyasak, Y., Alamdari, N.E., Gauthier, L., Grangier, P., Lemaitre, P. and Hamilton, W.L., 2023. **Bombardier aftermarket demand forecast with machine learning.** *INFORMS Journal on Applied Analytics*, 53(6), pp.425-445.

**Code block description**: We first install `darts` (this must be run every time a new colab notebook is open).

In [ ]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.3/963.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.9 MB/s eta 0:00:00
  

## Data inputs

This code block primarily  sets up the environment for time series analysis using models available in `darts`.

*Notable remarks:*

*   `data = pd.read_csv(..., index_col='ds')`: Reads time series data from a CSV file located at the given URL. It assumes the file has a column named 'ds' that represents the dates or timestamps and sets it as the index of the DataFrame.

*   `y = data['y']`: Extracts the time series data (likely the demand values) from the 'y' column of the DataFrame and assigns it to the variable y.

*   `plot_n_points = 104 `: Sets the number of data points to be used for plotting.

*   `test_n_points = 65`: Sets the number of data points to be used for testing the forecasting models.

In [ ]:
import darts
from darts.models import AutoARIMA, Prophet, ExponentialSmoothing, Theta, NaiveSeasonal, Croston, NBEATSModel
from darts.models import NaiveEnsembleModel, RegressionEnsembleModel
from darts.utils.utils import ModelMode, SeasonalityMode
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, smape, mase, rmsse, rmse, merr
from darts import TimeSeries

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# read historical time-series data (demand)
data = pd.read_csv('https://bit.ly/m5simple', index_col='ds')
data.index = pd.to_datetime(data.index)
n_periods = len(data.index)

# obtain the series of time series from column 'y'
y = data['y']

# number of historical data points to show on the plots (in weeks)
plot_n_points = 104

# number of data points used as the test set
test_n_points = 65


## Define key functions

This code block defines the following functions that will be called often in the pipeline.

**Function** `calculate_test_errors`:

This function calculates and return various error metrics for a time series forecast.

Input:

`test_series`: The actual values of the time series during the testing period.
`forecast_series`: The predicted values generated by a forecasting model during the testing period.

Process:

It uses functions from the darts.metrics module (`mape, smape, rmsse, rmse, merr`) to calculate the following error metrics (see [Darts error metrics](https://unit8co.github.io/darts/generated_api/darts.metrics.html)):


* MAPE: Mean Absolute Percentage Error (from function `mape`)
* sMAPE: Symmetric Mean Absolute Percentage Error (from function `smape`)
* RMSSE: Root Mean Squared Scaled Error (from function `rmsse`)
* RMSE: Root Mean Squared Error (from function `rmse`)
* ME: Mean Error (from function `merr`)
* Norm_ME: Mean Error normalized by the mean of actual demand

Output:
The function returns the errors DataFrame containing the calculated error metrics for the model.

**Function** `add_ranks`:

This function is used to add rank columns to a DataFrame based on the values in existing columns, It returns the modified DataFrame with the added rank columns.

In [ ]:
# functions to return test errors

def calculate_test_errors(test_series, forecast_series, series_name = 'y', print_results = False, skip_mape = False):
  # get the model name (the series from dart include the name of the model as the name of the series)
  m_name = forecast_series.name

  # convert the series of test set (actual values) and forecast set (forecasted values) into darts TimeSeries objects
  # (since we will use darts methods to calculate the errors)
  test = TimeSeries.from_series(test_series)
  forecast = TimeSeries.from_series(forecast_series)

  # In some cases, we need to skip MAPE if we have a zero demand observation since we cannot divide the error by zero
  # this is currently done manually. In any case, darts will also return an error if there is a zero demand when calculating mape.
  # The following part calculates all the forecasting errors we want to measure
  if not skip_mape:
    m_mape = mape(test, forecast)
    m_smape = smape(test, forecast)

  m_rmsse = rmsse(test, forecast, insample = train)
  m_rmse = rmse(test, forecast)
  m_merr = merr(test, forecast)

  if print_results:
    if not skip_mape:
      print(f"model {m_name} obtains Mean Absolute Percentage Error: {m_mape:.2f}%")
      print(f"model {m_name} obtains symmetric Mean Absolute Percentage Error: {m_smape:.2f}%")

    print(f"model {m_name} obtains Root Mean Squared Scaled Error: {m_rmsse:.2f}")
    print(f"model {m_name} obtains Root Mean Squared Error: {m_rmse:.2f}")
    print(f"model {m_name} obtains Mean Error: {m_merr:.2f}")

  # Now we put together the errors in the dataframe 'errors'=
  test_mean = test.pd_series().mean()

  if not skip_mape:
    column_names=['SeriesName', 'Model', 'MAPE', 'sMAPE', 'RMSSE', 'RMSE', 'Norm_RMSE', 'ME', 'Norm_ME']
    errors = pd.DataFrame(columns=column_names)
    errors.loc[0] = [series_name, m_name, m_mape, m_smape, m_rmsse, m_rmse, m_rmse/test_mean, m_merr, m_merr/test_mean]
  else:
    column_names=['SeriesName', 'Model', 'RMSSE', 'RMSE', 'Norm_RMSE', 'ME', 'Norm_ME']
    errors = pd.DataFrame(columns=column_names)
    errors.loc[0] = [series_name, m_name, m_rmsse, m_rmse, m_rmse/test_mean, m_merr, m_merr/test_mean]

  return errors

In [ ]:
# This function takes a DataFrame of the error values and then return the ranking of the results based on the obsolute values
# (the lower the value the higher the rank)
def add_ranks(df, ascending=True):
  for col in df.columns[2:]:
    df['Rank_'+col] = df[col].abs().rank(ascending=ascending)
  return df

## Pipeline 1: Forecasting and evaluation using two-fold validation

The code block calls and evaluates various time series forecasting models in the `darts` library. It first prepares the data, visualizes the actual values, and defines a list of forecasting models, including ensembles. Then, it iteratively trains each model, generates forecasts, calculates error metrics. Finally, it combines and displays the errors and ranks of each model, providing a performance comparison based on different models for the given dataset.

The training and evaluation of this pipeline is based on a two-fold validation process where data is split only into one training and one test set at a given split point. The predictions are evaluated against the actual values in the test set (which is not used in the training process).


In [ ]:
# Single split (two-fold)

# Create a plot the time-series and plot the actual values (both the train and test)
plt.figure(figsize=(18,6))
sns.scatterplot(x = y[-plot_n_points:].index, y = y[-plot_n_points:].values, label = 'true')

# Convert the series y into Darts's TimeSeries object (so that we can use darts methods)
y_timeseries = TimeSeries.from_series(y)

# We will also try an ensemble model that combines the foorecast of the following, methods
ensemble_models = [NaiveSeasonal(K=52), Theta(season_mode = SeasonalityMode.ADDITIVE), Croston()]

# Here is the list of the models (all supported by darts) and their corresponding input parameters we will run
list_models = [NaiveSeasonal(K=52), \
               ExponentialSmoothing(seasonal = None), \
               AutoARIMA(), \
               Theta(), \
               # Prophet(), \
               # NBEATSModel(input_chunk_length=52, output_chunk_length=52, n_epochs=50), \
               Croston(),\
               NaiveEnsembleModel(forecasting_models=ensemble_models), \
               RegressionEnsembleModel(forecasting_models=ensemble_models, regression_train_n_points=13)]

# We will prepare the list of errors that will contain the errors from each model to compare.
# This errors will be put together in a dataframe 'pd_errors' later on
list_errors = []

# We will also prepare a DataFrame that contains all the forecasts
df_forecasts = pd.DataFrame()

# We first add the first column based on the actual data
df_forecasts['y'] = y

# 'start' here define the split between the training and test periods
start = len(y)-test_n_points
# Now we split the train and test at the start point (of the test)
train, test = y_timeseries.split_before(start)

# Now we perform the forecast using one model in each loop, and calculate the forecasting errors
# The results are then added to the DataFrame 'df_forecasts'
for model in list_models:
  # First, we use the first 15 characters of the name of the model as the name to be displayed to DataFrame
  # (some names are too long so we cut only the first 15 characters)
  m_name = str(model)[:15]
  print("Forecast using model: ", m_name)

  # Train (fit) the mode using the training set
  model.fit(train)

  # Predict for the next 'test_n_points' and then convert the outputs (which is Darts TimeSeries object) into a series
  forecast_series = model.predict(test_n_points).pd_series()
  # Set the name of the series to the name of the model
  forecast_series.name = m_name
  # Now call the function to calculate all the errors
  errors = calculate_test_errors(test.pd_series(), forecast_series, series_name = 'y')
  # Add the forecast plot from this model to the time-series plot
  sns.lineplot(data = forecast_series, label = forecast_series.name)
  # Append the errors (list) into 'list_errors'
  list_errors.append(errors)
  # Add the forecast to 'df_forecasts' that store the forecasts of all the models
  df_forecasts[forecast_series.name] = forecast_series

# Once finished, we then convert the list of list of errors into a DataFrame
pd_errors = pd.concat(list_errors, ignore_index = True)

# We then add the columns to show the rankings based on the performances of the forecasts of different models
pd_errors = add_ranks(pd_errors)

pd_errors

## Define rolling forecasts and plot functions

**Function** `eval_model_rolling_forecasts`:

This function simulates a real-world forecasting scenario where one would retrain the model periodically with new data to make future predictions. It provides a more realistic evaluation of the model's performance over time compared to a single train-test split. This functions requires the **following inputs**:

* `model`: darts models (see [darts forecasting models](https://unit8co.github.io/darts/generated_api/darts.models.forecasting.html))
* `series`: a series containing the entire time-series data
* `start`: the first starting point of the forecasting evaluation
* `fh`: length of the forecast horizon
* `stride`: the retraining (forecast update) internal (i.e., new forecasts are produced every `stride` periods). This value must be $\leq$ `fh`

The model returns a series containing the predictions that are produced using a rolling update of every `stride` periods. Only the most recent values of the predictions are kept in the series.

**Function** `plot_forecasts`:

 This function is used to visualize time series data from a DataFrame containing actual and forecasted data.It generates a plot with actual values displayed as scatter points and forecasts from different models represented as lines.

In [ ]:

def eval_model_rolling_forecasts(model, series, start, fh, stride, series_name = 'y'):
  m_name = str(model)[:15]

  timeseries = TimeSeries.from_series(series)
  list_hist_rolling_forecasts = []
  for starting_point in range(start, len(timeseries), stride):
    train, val = timeseries.split_before(starting_point)
    forecast_horizon = min(fh, len(val))
    forecasts = model.fit(train).predict(forecast_horizon)
    list_hist_rolling_forecasts.append(forecasts.pd_series())

  last_fold = len(list_hist_rolling_forecasts)-1
  hist_rolling_forecasts = list_hist_rolling_forecasts[last_fold]

  for fold in range(last_fold):
    hist_rolling_forecasts = pd.concat([hist_rolling_forecasts, list_hist_rolling_forecasts[fold][:stride]])

  hist_rolling_forecasts.name = m_name
  print("\t----- generating rolling forecasts using model " +str(hist_rolling_forecasts.name))

  return hist_rolling_forecasts


In [ ]:
def plot_forecasts(df_forecasts, n_points, series_name = 'y', plot_name = 'Prediction'):
  plt.figure(figsize=(18,6))
  plt.title(plot_name+": " + series_name)
  plt.xlabel("Date")
  plt.ylabel("Value")
  y = df_forecasts['y']
  sns.scatterplot(x = y[-n_points:].index, y = y[-n_points:].values, label = 'true')

  for model in df_forecasts.columns[1:]:
    sns.lineplot(data = df_forecasts[model][-n_points:], label = model)


## Pipeline 2: Forecasting and evaluation using sequential validation

This code block is similar to pipeline 1 but it simulates and evaluates the forecasts on a rolling basis using the previously defined function `eval_model_rolling_forecasts(...)`.

In [ ]:
# rolling forecasts evaluation (multifold sequential validation)

val_size = test_n_points
y_timeseries = TimeSeries.from_series(y)
start = len(y)-val_size
fh = 13
stride=13

ensemble_models = [NaiveSeasonal(K=52), Theta(season_mode = SeasonalityMode.ADDITIVE), Croston()]

list_models = [NaiveSeasonal(K=52),
               ExponentialSmoothing(),
               AutoARIMA(), \
               Theta(), \
               # Prophet(), \
               # NBEATSModel(input_chunk_length=52, output_chunk_length=52, n_epochs=50), \
               Croston(),\
               NaiveEnsembleModel(forecasting_models=ensemble_models), \
               RegressionEnsembleModel(forecasting_models=ensemble_models, regression_train_n_points=13)
               ]

list_errors = []
df_forecasts = pd.DataFrame()
df_forecasts['y'] = y
train, test = y_timeseries.split_before(start)

# Now we perform the forecast using one model in each loop, and calculate the forecasting errors
# The results are then added to the DataFrame 'df_forecasts'
for model in list_models:
  forecast_series = eval_model_rolling_forecasts(model, y, start, fh, stride)
  errors = calculate_test_errors(test.pd_series(), forecast_series)
  list_errors.append(errors)
  df_forecasts[forecast_series.name] = forecast_series

pd_errors = pd.concat(list_errors)

# We also use our previously defined function 'add_ranks' to
pd_errors = add_ranks(pd_errors)

plot_forecasts(df_forecasts, plot_n_points)

pd_errors

## Pipeline 3: Automating forecasting and sequential validation on multiple items

This code block is similar to pipeline 2 that performs rolling forecasts. The code automate the predictions of multiple items through iterative process.

In [ ]:
# pipeline to run forecast for multiple items

import time

# state-item level
url = 'https://raw.githubusercontent.com/acedesci/scanalytics/refs/heads/master/EN/data/M5/processed_data/m5data_CA_item_state_rand.csv'

# store-item level (slow-moving)
# url = 'https://raw.githubusercontent.com/acedesci/scanalytics/refs/heads/master/EN/data/M5/processed_data/m5data_CA_item_store_n25_cv.csv'

data = pd.read_csv(url, index_col='ds')
data.index = pd.to_datetime(data.index)

list_items = data.columns[:3]
print("list of items: " + str(list_items))

val_size = test_n_points
start = len(data.index)-val_size
fh = 13
stride = 13

ensemble_models = [NaiveSeasonal(K=52), Theta(season_mode = SeasonalityMode.ADDITIVE), Croston()]

list_models = [NaiveSeasonal(K=52),
               # ExponentialSmoothing(),
               AutoARIMA(), \
               Theta(season_mode = SeasonalityMode.ADDITIVE), \
               # Prophet(), \
               # NBEATSModel(input_chunk_length=52, output_chunk_length=52, n_epochs=50), \
               Croston(),\
               NaiveEnsembleModel(forecasting_models=ensemble_models), \
               RegressionEnsembleModel(forecasting_models=ensemble_models, regression_train_n_points=13)
               ]

pd_all_errors = pd.DataFrame()

list_forecasts = []

# Iterate through each item (time-series) in the list of items
for item in list_items:

  print("Item: "+str(item))
  y = data[item]
  df_forecasts = pd.DataFrame()
  df_forecasts['y'] = y

  y_timeseries = TimeSeries.from_series(y)
  train, test = y_timeseries.split_before(start)

  list_errors = []

  # for each item, perform the forecast using one model in each loop, and calculate the forecasting errors
  # the results are then added to the DataFrame 'df_forecasts'
  for model in list_models:
    start_time = time.time()
    forecast_series = eval_model_rolling_forecasts(model, y, start, fh, stride, series_name = item)
    runtime = time.time()-start_time
    forecast_series = forecast_series.clip(lower=0)
    errors = calculate_test_errors(test.pd_series(), forecast_series, series_name = item, skip_mape = True)
    errors['runtime'] = [runtime]
    list_errors.append(errors)
    df_forecasts[forecast_series.name] = forecast_series

  pd_errors = add_ranks(pd.concat(list_errors))

  pd_all_errors = pd.concat([pd_all_errors, pd_errors])
  list_forecasts.append(df_forecasts)

In [ ]:
# show the errors from all the models for each item
pd_all_errors

In [ ]:
# show the average errors from all the models across all the items
pd_all_errors[pd_all_errors.columns[1:]].groupby('Model').mean().sort_values(by=['Rank_RMSSE'])

In [ ]:
for i, item in enumerate(list_items[:10]):
  plot_forecasts(list_forecasts[i], plot_n_points, series_name = item)

## Extra: Dynamic inventory policy from forecasts

The code calculates a dynamic basestock inventory policy using previously generated demand forecasts from various models. In this basestock inventory policy, we assume that the stock must be replenished to the target level ($S_t$) at every replenishment period. This target stock level must ensure that the inventory position (on-hand inventory + in-transit inventory) is sufficient to cover the forecasted demands and safety stock during the supply lead time. More specifically, the dynamic basestock (target) inventory level at time $t$, denoted by $S_t$, is calculated by:

$S_t = \sum_{k=t+1}^{t+L}F_k+SS_t$

where $F_t$ is the forecast of the item at time $t$, $L$ is the lead time, and $SS_t$ is the dynamic safety stock at time $t$ calculated as follows:

$SS_t = z\times RMSE \times \sqrt{L}$

where $RMSE$ is the standard deviation of the forecasting errors (i.e., root mean squared error). This value is calculated from the reference (initial) periods prior to rolling forecasts of that iteration defined by the parameter `n_periods_fcst_errors`. The evaluation of the inventory policy starts from the period following the end of the initial periods.

Finally, the code visualizes both the original demand forecasts and the calculated target stock levels against the ideal target stock levels based on the actual demands.

In [ ]:
# dynamic basestock policy simulation

import numpy as np

item_index = 2

forecasts = list_forecasts[item_index][-test_n_points:]
forecast_models = forecasts.columns[1:]

n_periods_fcst_errors = 13

leadtime = 3
z_score = 1.96

test_forecasts = forecasts[n_periods_fcst_errors:]

# prepare the dataframe for the target stock values for each time period that will be calculated
target_stocks = pd.DataFrame(index = test_forecasts.index, columns = test_forecasts.columns)

hist_rmse = []

list_errors = []

# calculate the (dynamic) target stock levels for each period in the test periods
for starting_period in range(len(test_forecasts.index)-leadtime+1):
  initial_forecasts = forecasts[starting_period:starting_period+n_periods_fcst_errors]

  actual_leadtime_demand = sum(test_forecasts['y'].iloc[starting_period: starting_period+leadtime])
  target_stocks['y'].iloc[starting_period] = actual_leadtime_demand

  for model in forecast_models:
    squared_diff = (initial_forecasts['y'] - initial_forecasts[model])**2
    rmse_val = (np.mean(squared_diff))**(1/2)

    exp_leadtime_demand = sum(test_forecasts[model].iloc[starting_period: starting_period+leadtime])
    safety_stock = z_score*rmse_val*(leadtime)**(1/2)
    target_stocks[model].iloc[starting_period] = exp_leadtime_demand + safety_stock
    # print("Model["+str(model)+"], SS: "+str(safety_stock))

plot_forecasts(list_forecasts[item_index], plot_n_points)
plot_forecasts(target_stocks, plot_n_points, plot_name = 'Target stocks')


In [ ]:
# Retrieve and display the forecasting performances for this item based on different models
item = list_items[item_index]
pd_all_errors[pd_all_errors['SeriesName'] == item]

In [ ]:
# save the files needed for the simulation (will be performed in a separate notebook)

from google.colab import drive
drive.mount('/content/drive')
test_forecasts.to_csv('/content/drive/My Drive/test_forecasts.csv')
target_stocks.to_csv('/content/drive/My Drive/target_stocks.csv')